## feauterization

Lo que hago es feauterizo, normalizo y despues lo mando como un json para poder hacerlo csv y despues importarlo facil


### Como es la normalizacion que estoy usando?

Bueno lo que hago es hacer que todos las features sean normalizadas entre 0 1, para que despues durante el entrenamiento cuando se concatene al output de las convoluciones se escale a sus respectivos maximos y minimos, de cada compuesto.

Esta idea la saco de como funciona el minmax scaleing
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

```python
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # esto lo hago afuera, pq siempre es lo mismo para cada vez que se usa el vector de features
X_scaled = X_std * (max - min) + min # este es el paso que hago adentro de la red, en base a cada output de las covoluciones
```

Se hace en dos pasos, ese Scaleing: primero se genera que todo el vector este entre 01 y despues se le improntan los maximos y minimos de los outputs de las convolucinoes.

In [3]:
from deepchem.feat import RDKitDescriptors
import pandas as pd
import numpy as np
import json
import os

def add_column_feturized_json(path_original,output_path):
    namefile = path_original.split("/")[-1]
    name = namefile[:namefile.find(".csv")]
    featurizer = RDKitDescriptors()     
    df = pd.read_csv(path_original)
    rdkit_features_not_std = featurizer.featurize(df["smiles"])
    remove_nan = ~((np.isnan(rdkit_features_not_std)).any(axis=1))
    rdkit_features_not_std = rdkit_features_not_std[remove_nan]
    df_output = df[remove_nan] 
    comps_droped = len(df) - len(rdkit_features_not_std)
    print("problematic compounds droped:",comps_droped)
    rdkit_features_not_std_max = rdkit_features_not_std.max(axis=0,keepdims=True)
    rdkit_features_not_std_min = rdkit_features_not_std.min(axis=0,keepdims=True)
    rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)
    # ESTO ES UNA INCOSISTENCIA BRUTAL PORQUE VAN A HABER DISTINTAS FEATURES EN CADA TARGET...
    rdkit_features = rdkit_features[:,(~np.isnan(rdkit_features)).all(axis=0)]
    
    np.save(os.path.join(output_path,(name+"features_rdkit.npy")),rdkit_features)
    df_output.to_csv(os.path.join(output_path,(name+"features_rdkit.csv")),index=False)


In [4]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL262.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 4


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [5]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL4657.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 0


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [12]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL4072.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 1


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [13]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 0


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


trials de splting

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850features_rdkit.csv")
features = np.load("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850features_rdkit.npy")

In [5]:
data

,comp_id,bioactivity,data_split,smiles
0,CHEMBL101747,1,test,CCN1CCC(CC(=O)Nc2n[nH]c3nc(-c4cccs4)c(Br)cc23)CC1
1,CHEMBL101804,1,train,O=C(Nc1n[nH]c2nc(-c3ccco3)ccc12)C1CC1
2,CHEMBL102714,1,train,Cn1cc(C2=C(c3ccc(Cl)cc3Cl)C(=O)NC2=O)c2ccccc21
3,CHEMBL1078178,0,test,N#CCNC(=O)c1ccc(-c2ccnc(Nc3ccc(N4CCOCC4)cc3)n2...
4,CHEMBL1079175,0,test,NC1(c2ccc(-c3nc4ccn5c(=O)[nH]nc5c4cc3-c3ccccc3...
...,...,...,...,...
696,CHEMBL607707,0,train,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...
697,CHEMBL608533,1,train,CO[C@@H]1[C@H](N(C)C(=O)c2ccccc2)C[C@H]2O[C@]1...
698,CHEMBL63803,1,train,COc1cc(/C=N/Nc2ncnc3c2cnn3-c2ccccc2)ccc1O
699,CHEMBL75680,1,train,CCCCc1c(-c2ccc(O)cc2)[nH]c2nccnc12


In [11]:
data[data["data_split"]=="train"]

,comp_id,bioactivity,data_split,smiles
1,CHEMBL101804,1,train,O=C(Nc1n[nH]c2nc(-c3ccco3)ccc12)C1CC1
2,CHEMBL102714,1,train,Cn1cc(C2=C(c3ccc(Cl)cc3Cl)C(=O)NC2=O)c2ccccc21
5,CHEMBL1080901,1,train,Nc1nc(NCCNc2ncc(-c3ncc[nH]3)c(-c3ccc(Cl)cc3Cl)...
6,CHEMBL1082152,1,train,Cn1cc(C2=C(c3cccc(NCC(O)CO)c3)C(=O)NC2=O)c2cc(...
17,CHEMBL1201182,0,train,CO[C@H]1C[C@@H]2CC[C@@H](C)[C@@](O)(O2)C(=O)C(...
...,...,...,...,...
696,CHEMBL607707,0,train,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...
697,CHEMBL608533,1,train,CO[C@@H]1[C@H](N(C)C(=O)c2ccccc2)C[C@H]2O[C@]1...
698,CHEMBL63803,1,train,COc1cc(/C=N/Nc2ncnc3c2cnn3-c2ccccc2)ccc1O
699,CHEMBL75680,1,train,CCCCc1c(-c2ccc(O)cc2)[nH]c2nccnc12


In [10]:
len(features[data["data_split"]=="train"])

448

In [4]:
len(features)

701

In [14]:
len(features[0])

193